# PROJECT

- intro
- point 2
- point 3

## Notes for next project
- Visualization
- Documentation
- Plan the project better
- Replace manual steps using more and more functions (data cleaning)
- Study and understand topic better
- Add analysis and learnings after each visual


## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
...
...
...
...
...

Ellipsis

## Importing Data

In [2]:
#data = pd.read_csv(r"C:\users\nicol\lab-cleaning-numerical-data\files_for_lab\we_fn_use_c_marketing_customer_value_analysis.csv")

## Reviewing Data

In [3]:
data.shape

NameError: name 'data' is not defined

In [4]:
data.info()

NameError: name 'data' is not defined

In [ ]:
data.dtypes

In [ ]:
# add comments and learnings after each step - e.g. here I could add what dtypes might be critical

In [ ]:
data.describe

## Data cleaning

### Standardize

In [5]:
df.columns = [column.lower().replace(' ', '_') for column in df.columns]

NameError: name 'df' is not defined

### Categorial data

In [6]:
cat = df1.select_dtypes('object')
cat

NameError: name 'df1' is not defined

### Numerical data

### Null and zero values

In [ ]:
data.isnull().sum().sort_values(ascending=False)

### Drop columns

In [ ]:
data.drop('director', axis=1)

## Drop Duplicates

In [ ]:
#drop_duplicates: data duplicated because of disagreement in releaase_year
df_netflix.drop_duplicates(['title'], keep='first', inplace=True)

## Outliers

## Data Imbalances

## Normalize Data

## Model

## Evaluate Quality of the Model

## Try something else

## Visualizations

## Conclusion